# 神经网络基本概念

神经网络基本概念可以参考：[深度学习 Deep Learning](https://space.bilibili.com/88461692/channel/detail?cid=26587)。这里对一些实践中会碰到的概念做些补充，逐渐添加中...

## Loss and Loss Functions

本节参考了：[Loss and Loss Functions for Training Deep Learning Neural Networks](https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/)。

loss和loss函数很平常了，不过对于其在nn中的角色和如何选择合适的还是需要了解下。

## 神经网络简单实现

用numpy手写神经网络。

In [ ]:
import numpy as np
import os
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from matplotlib import cm

# 首先是标定神经网络的结构
NN_ARCHITECTURE = [{"input_dim": 2, "output_dim": 25, "activation": "relu"},
                   {"input_dim": 25, "output_dim": 50, "activation": "relu"},
                   {"input_dim": 50, "output_dim": 50, "activation": "relu"},
                   {"input_dim": 50, "output_dim": 25, "activation": "relu"},
                   {"input_dim": 25, "output_dim": 1, "activation": "sigmoid"}]


def init_layers(nn_architecture, seed=99):
    """然后对权重进行初始化，为了保证每次计算的结果一致，初始化种子使用同样的值"""
    # 首先随机初始化种子
    np.random.seed(seed)
    # 初始化关键变量
    number_of_layers = len(nn_architecture)
    params_values = {}
    for idx, layer in enumerate(nn_architecture):
        layer_idx = idx+1
        layer_input_size = layer["input_dim"]
        layer_output_size = layer["output_dim"]
        # 对每一层，初始化W矩阵和向量b。要注意矩阵和向量的维度，因为
        params_values['W'+str(layer_idx)] = np.random.randn(
            layer_output_size, layer_input_size)*0.1
        params_values['b'+str(layer_idx)
                      ] = np.random.randn(layer_output_size, 1)*0.1
    return params_values


def sigmoid(Z):
    """sigmoid激活函数"""
    return 1/(1+np.exp(-Z))


def sigmoid_backward(dA, Z):
    """sigmoid函数导数在反向传播时的作用"""
    sig = sigmoid(Z)
    return dA*sig*(1-sig)


def relu(Z):
    """relu激活函数"""
    return np.maximum(0, Z)


def relu_backward(dA, Z):
    """relu函数导数"""
    # copy=True意味着构建了一个数组dA的副本，即深拷贝
    dZ = np.array(dA, copy=True)
    # 直接令z<=0时的dz为0即可，剩下的相当于dA*1
    dZ[Z <= 0] = 0
    return dZ


def single_layer_forward_propagation(A_prev, W_curr, b_curr, activation="relu"):
    """单层前向传递函数"""
    Z_curr = np.dot(W_curr, A_prev)+b_curr
    if activation is "relu":
        activation_func = relu
    elif activation is "sigmoid":
        activation_func = sigmoid
    else:
        raise Exception("Non-supported activation function")
    return activation_func(Z_curr), Z_curr


def full_forward_propagation(X, params_values, nn_architecture):
    """所有层前向传递"""
    # 构建一个变量存储后续反向传播所需的各层信息
    memory = {}
    # X是输入
    A_curr = X
    for idx, layer in enumerate(nn_architecture):
        # layer_idx从1开始计数
        layer_idx = idx+1
        A_prev = A_curr
        activ_function_curr = layer["activation"]
        W_curr = params_values["W"+str(layer_idx)]
        b_curr = params_values["b"+str(layer_idx)]
        A_curr, Z_curr = single_layer_forward_propagation(
            A_prev, W_curr, b_curr, activ_function_curr)
        # memory中存储中间变量
        memory["A"+str(idx)] = A_prev
        memory["Z"+str(layer_idx)] = Z_curr
    return A_curr, memory


def get_cost_value(Y_hat, Y):
    m = Y_hat.shape[1]
    cost = -1/m*(np.dot(Y, np.log(Y_hat).T)+np.dot(1-Y, np.log(1-Y_hat).T))
    # 为什么要除去多余的维度？因为之前的计算导致的
    return np.squeeze(cost)


def single_layer_backward_propagation(dA_curr, W_curr, b_curr, Z_curr, A_prev, activation="relu"):
    """计算一层误差反向传播，输入dA_curr表示的是∂J/∂a^(l)，其中，l是指当前层。
    输出是损失函数对上一层l-1层神经元值以及本层l层权重和偏置的偏导"""
    # m表示的是样本个数
    m = A_prev.shape[1]
    if activation is "relu":
        backward_activation_func = relu_backward
    elif activation is "sigmoid":
        backward_activation_func = sigmoid_backward
    else:
        raise Exception('Non-supported activation function')
    # 因为每个偏导都包含dZ_curr，因此先求它。dZ_curr表示的是∂J/∂Z^(l)，链式法则=∂J/∂a^(l) * f'(Z^(l))（对应项相乘）
    dZ_curr = backward_activation_func(dA_curr, Z_curr)
    # 接下来几项就是三个偏导，注意有样本的影响，维度和单一样本推导时有所不同
    dW_curr = np.dot(dZ_curr, A_prev.T)/m
    db_curr = np.sum(dZ_curr, axis=1, keepdims=True)/m
    dA_prev = np.dot(W_curr.T, dZ_curr)
    return dA_prev, dW_curr, db_curr


def full_backward_propagation(Y_hat, Y, memory, params_values, nn_architecture):
    """完整反向传播算法，给出所有参数偏导组成的完整梯度"""
    grads_values = {}
    # 在反向循环开始之前，给出最后输出的误差，初始化反向传播算法
    m = Y.shape[1]
    Y = Y.reshape(Y_hat.shape)
    dA_prev = -(np.divide(Y, Y_hat)-np.divide(1-Y, 1-Y_hat))
    # 因为是反向传播算法，因此循环顺序是从后往前
    for layer_idx_prev, layer in reversed(list(enumerate(nn_architecture))):
        layer_idx_curr = layer_idx_prev+1
        activ_function_curr = layer["activation"]

        dA_curr = dA_prev

        A_prev = memory["A"+str(layer_idx_prev)]
        Z_curr = memory["Z"+str(layer_idx_curr)]
        W_curr = params_values["W"+str(layer_idx_curr)]
        b_curr = params_values["b"+str(layer_idx_curr)]

        dA_prev, dW_curr, db_curr = single_layer_backward_propagation(
            dA_curr, W_curr, b_curr, Z_curr, A_prev, activ_function_curr)

        grads_values['dW'+str(layer_idx_curr)] = dW_curr
        grads_values['db'+str(layer_idx_curr)] = db_curr
    return grads_values


def update(params_values, grads_values, nn_architecture, learning_rate):
    """更新参数"""
    for layer_idx, layer in enumerate(nn_architecture, 1):
        params_values['W'+str(layer_idx)] -= learning_rate * \
            grads_values['dW'+str(layer_idx)]
        params_values['b'+str(layer_idx)] -= learning_rate * \
            grads_values['db'+str(layer_idx)]
    return params_values


def convert_prob_into_class(probs):
    probs_ = np.copy(probs)
    probs_[probs_ > 0.5] = 1
    probs_[probs_ <= 0.5] = 0
    return probs_


def get_accuracy_value(y_hat, y):
    """为了显示优化的过程，计算accuracy"""
    y_hat = convert_prob_into_class(y_hat)
    return (y_hat == y).all(axis=0).mean()


def train(X, Y, nn_architecture, epochs, learning_rate, verbose=False, callback=None):
    """整体训练过程"""
    # 初始化神经网络参数
    param_values = init_layers(nn_architecture, 2)
    # 为了后续可视化优化过程，存储中间阶段优化结果
    cost_history = []
    accuracy_history = []
    # 按epoch循环训练神经网络
    for i in range(epochs):
        # 每一步里面，首先前向传播
        Y_hat, cashe = full_forward_propagation(
            X, param_values, nn_architecture)
        # 然后计算cost
        cost = get_cost_value(Y_hat, Y)
        cost_history.append(cost)
        accuracy = get_accuracy_value(Y_hat, Y)
        accuracy_history.append(accuracy)
        # 然后反向传播
        grads_values = full_backward_propagation(
            Y_hat, Y, cashe, param_values, nn_architecture)
        # 得到梯度之后，更新参数
        param_values = update(param_values, grads_values,
                              nn_architecture, learning_rate)
        # 显示训练过程
        if (i % 50 == 0):
            if (verbose):
                print(
                    "Iteration:{:05}-cost:{:.5f}-accuracy:{:.5f}".format(i, cost, accuracy))
            if (callback is not None):
                callback(i, param_values)
    return param_values


# number of samples in the data set
N_SAMPLES = 1000
# ratio between training and test sets
TEST_SIZE = 0.1
X, y = make_moons(n_samples=N_SAMPLES, noise=0.2, random_state=100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=42)


def make_plot(X, y, plot_name, file_name=None, XX=None, YY=None, preds=None, dark=False):
    """可视化数据集的函数"""
    plt.figure(figsize=(16, 12))
    axes = plt.gca()
    axes.set(xlabel="$X_1$", ylabel="$X_2$")
    plt.title(plot_name, fontsize=30)
    plt.subplots_adjust(left=0.20)
    plt.subplots_adjust(right=0.80)
    if(XX is not None and YY is not None and preds is not None):
        plt.contourf(XX, YY, preds.reshape(XX.shape),
                     25, alpha=1, cmap=cm.Spectral)
        plt.contourf(XX, YY, preds.reshape(XX.shape),
                     levels=[.5], cmap="Greys", vmin=0, vmax=.6)
    plt.scatter(X[:, 0], X[:, 1], c=y.ravel(), s=40,
                cmap=plt.cm.Spectral, edgecolors='black')
    plt.show()
    if(file_name):
        plt.savefig(file_name)
        plt.close()


make_plot(X, y, "Dataset")
# Training
params_values = train(np.transpose(X_train), np.transpose(
    y_train.reshape((y_train.shape[0], 1))), NN_ARCHITECTURE, 10000, 0.01)
# Prediction
Y_test_hat, _ = full_forward_propagation(
    np.transpose(X_test), params_values, NN_ARCHITECTURE)
# Accuracy achieved on the test set
acc_test = get_accuracy_value(Y_test_hat, np.transpose(
    y_test.reshape((y_test.shape[0], 1))))
print("Test set accuracy: {:.2f} - David".format(acc_test))

## 模型训练可视化

从这里开始补充一些除了基本概念之外也常需要了解的内容。首先看看一次简单训练过程中，如何进行loss的可视化等

先看看下面程序的基本流程，首先是数据加载，然后是分割成训练集和测试集，然后构造模型，接下来就是训练和展示结果。

这里首先说下fit函数的返回值会返回些什么内容。history是一个tensorflow中的keras的对象。其中有：

- epoch: 全部epoch的编号，从0开始计数
- history: loss和val_loss，都是各个epoch的值，分别对应训练集和验证集
- model: 模型整理结构，也包括权重数据
- params: 包括batch_size和epochs等各类模型参数。

In [ ]:
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
history = model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, verbose=0)  # list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## 交叉验证

交叉验证是模型调优过程中，很重要的一个环节。这里参考了：[模型调优：交叉验证，超参数搜索(复习17)](https://blog.csdn.net/cymy001/article/details/79120355)以及[交叉验证](https://www.jiqizhixin.com/graph/technologies/de51a6aa-a6c6-43b1-880f-79c0b033bbae)来记录一点基本概念。

用模型在测试集上进行性能评估前，通常是希望**尽可能利用手头现有的数据对模型进行调优**，甚至可以**粗略地估计测试结果**。

交叉验证，有时亦称循环估计， 是一种统计学上将数据样本切割成较小子集的实用方法。于是可以**先在一个子集上做分析， 而其它子集则用来做后续对此分析的确认及验证**。 一开始的子集被称为训练集。而其它的子集则被称为验证集或测试集。交叉验证的目标是定义一个数据集到“测试”的模型在训练阶段，**以便减少像过拟合的问题**，得到该模型将如何衍生到一个独立的数据集的提示。

交叉验证可以**保证所有数据都有被训练和验证的机会**，也尽最大可能让优化的模型性能表现的更加可信。

K次交叉验证（k-fold cross-validation）是一种常用的方式，将**训练集分割成K个子样本，一个单独的子样本被保留作为验证模型的数据，其他K-1个样本用来训练**。交叉验证重复K次，每个子样本验证一次，平均K次的结果或者使用其它结合方式，最终得到一个单一估测。这个方法的优势在于，同时重复运用随机产生的子样本进行训练和验证，每次的结果验证一次，10次交叉验证是最常用的。

交叉验证可用于**比较不同预测建模程序的性能**。注意它其实是用来判断模型性能以及选取参数的。因此确定好参数之后，就确定了模型，再重新用这个模型去训练预测就行了。

可以使用scikit-learn调用Keras生成的模型。

使用scikit-learn封装Keras的模型，使用scikit-learn对Keras的模型进行交叉验证，使用scikit-learn，利用网格搜索调整Keras模型的超参.

首先，Keras在深度学习很受欢迎，但是只能做深度学习：Keras是最小化的深度学习库，目标在于快速搭建深度学习模型。现在已经可以在tensorflow2中直接使用keras接口了。

另一方面，基于SciPy的scikit-learn，数值运算效率很高，适用于普遍的机器学习任务，提供很多机器学习工具，包括但不限于：

- 使用K折验证模型
- 快速搜索并测试超参

Keras为scikit-learn封装了KerasClassifier和KerasRegressor。

In [ ]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
import numpy
import pandas


# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
    model.add(Dense(8, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=150, batch_size=10)
# evaluate using 10-fold cross validation
# TODO： 这个StratifiedKFold已经废弃了，所以参考 from sklearn.model_selection import StratifiedKFold
kfold = StratifiedKFold(y=Y, n_folds=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

接下来用pima的实例数据展示交叉验证。大部分过程前面以及提到过了，这里说明下model.evaluate 函数的作用。evaluate函数返回the loss value & metrics values for the model in test mode，在没有特别指定的情况下，返回的是test模式下的loss。

In [ ]:
# MLP for Pima Indians Dataset with 10-fold cross validation
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import StratifiedKFold
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    model.fit(X[train], Y[train], epochs=150, batch_size=10, verbose=0)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
    cvscores.append(scores[1] * 100)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))

## 模型保存

本节主要参考了：[用序列化保存模型](https://cnbeining.github.io/deep-learning-with-python-cn/4-advanced-multi-layer-perceptrons-and-keras/ch13-save-your-models-for-later-with-serialization.html)。使用保存点保存最好的模型，每轮效果变好就保存一下，还是很好的。

In [ ]:
# Checkpoint the weights when validation accuracy improves
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
filepath = "weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, callbacks=callbacks_list, verbose=0)

只保存最好的模型，每次如果效果变好就覆盖之前的权重文件，把保存的文件名改成固定的即可

In [ ]:
# Checkpoint the weights for best model on validation accuracy
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# checkpoint
filepath = "weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,
                             mode='max')
callbacks_list = [checkpoint]
# Fit the model
model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10,
          callbacks=callbacks_list, verbose=0)

模型计算有时时间很长，所以需要保存相关计算结果，序列化是很好的方式

In [ ]:
# MLP for Pima Indians Dataset serialize to JSON and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

保存的模型自然也是可以重新加载的，保存的时候有model和weights两个部分，因此读取的时候也是两部分，并且在使用的时候还要注意加载的模型还必须compile之后，才能使用。这里以保存为yaml文件为例。

In [ ]:
# MLP for Pima Indians Dataset serialize to YAML and HDF5
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_yaml
import numpy
import os

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
# serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

# later...
# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
# load weights into new model loaded_model.load_weights("model.h5") print("Loaded model from disk")
loaded_model = model_from_yaml(loaded_model_yaml)
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1] * 100))


导入保存的模型，保存点只保存权重，网络结构需要预先保存

In [ ]:
# How to load and use weights from a checkpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))
# load weights
model.load_weights("weights.best.hdf5")
# Compile model (required to make predictions)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Created model and loaded weights from file")
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:, 0:8]
Y = dataset[:, 8]
# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))